In [ ]:
import time
import arkouda as ak
import arachne as ar

In [ ]:
ak.connect("n82", 5555)

In [ ]:
def get_gnp_random_graph(n, p, seed):
    G = ar.gnp_random_graph(n, p, create_using=ar.PropGraph, seed=seed)
    print(f"Built G(n,p) graph with {len(G):,} vertices and {G.size():,} edges.")

    return G

In [ ]:
def build_subgraph(src, dst):
    akarray_src = ak.array(src)
    akarray_dst = ak.array(dst)

    H = ar.PropGraph()
    H.add_edges_from(akarray_src, akarray_dst)

    return H

In [ ]:
def add_attributes(graph, num_node_lbls, num_edge_lbls, vals_per_lbl, seed):
    nodes = graph.nodes()
    edges = graph.edges()

    n = len(nodes)
    m = len(edges[0])

    node_attributes = []
    edge_attributes = []

    for i in range(num_node_lbls):
        node_attributes.append(ak.randint(0,vals_per_lbl,n,seed=seed*i,dtype=ak.int64))

    for i in range(num_edge_lbls):
        edge_attributes.append(ak.randint(0,vals_per_lbl,m,seed=seed*i,dtype=ak.int64))

    lbls_dict = {"lbl"+str(idx):attribute for idx, attribute in enumerate(node_attributes)}
    rels_dict = {"rel"+str(idx):attribute for idx, attribute in enumerate(edge_attributes)}

    node_dict = {"nodes" : nodes}
    edge_dict = {"src" : edges[0], "dst" : edges[1]}

    node_dict.update(lbls_dict)
    edge_dict.update(rels_dict)

    node_df = ak.DataFrame(node_dict)
    edge_df = ak.DataFrame(edge_dict)

    graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
    graph.load_node_attributes(node_df, node_column="nodes")

In [ ]:
n = 50_000
p = 0.0005
seed = 42
num_node_lbls = 2
num_edge_lbls = 2
vals_per_lbl = 2

In [ ]:
src = [0, 1, 0]
dst = [1, 2, 2]

In [ ]:
G = get_gnp_random_graph(n, p, seed)

In [ ]:
H = build_subgraph(src, dst)

In [ ]:
# Run structural subgraph isomorphism.
start = time.time()
results1 = ar.subgraph_isomorphism(G, H, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H):_} isos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_isomorphism(G, H, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H):_} isos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_isomorphism(G, H, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} isos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_isomorphism(G, H, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results3:_} isos with PS (count-only) took {end-start:.2f} seconds")

In [ ]:
# Run structural subgraph monomorphism.
start = time.time()
results1 = ar.subgraph_monomorphism(G, H, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H):_} monos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_monomorphism(G, H, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H):_} monos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_monomorphism(G, H, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} monos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_monomorphism(G, H, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results3:_} monos with PS (count-only) took {end-start:.2f} seconds")

In [ ]:
add_attributes(G, num_node_lbls, num_edge_lbls, vals_per_lbl, seed)

In [ ]:
add_attributes(H, num_node_lbls, num_edge_lbls, vals_per_lbl, seed)

In [ ]:
# Run semantic subgraph isomorphism.
start = time.time()
results1 = ar.subgraph_isomorphism(G, H, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H):_} isos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_isomorphism(G, H, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H):_} isos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_isomorphism(G, H, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} isos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_isomorphism(G, H, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results3:_} isos with PS (count-only) took {end-start:.2f} seconds")

In [ ]:
# Run semantic subgraph monomorphism.
start = time.time()
results1 = ar.subgraph_monomorphism(G, H, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H):_} monos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_monomorphism(G, H, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H):_} monos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_monomorphism(G, H, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} monos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_monomorphism(G, H, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results3:_} monos with PS (count-only) took {end-start:.2f} seconds")